In [1]:
#pip install poke-env
#pip install --upgrade poke-env
#pip show poke-env
#cd "C:\Users\Isac\Desktop\Pokémon project\pythonProject\pokemon-showdown"
#node pokemon-showdown start --no-security

In [2]:
import numpy as np
from poke_env.teambuilder import Teambuilder

class RandomTeamFromPool(Teambuilder):
    def __init__(self, teams):
        self.teams = [self.join_team(self.parse_showdown_team(team)) for team in teams]

    def yield_team(self):
        return np.random.choice(self.teams)
    

In [3]:
import os

teamsPaths = []
for root, dirs, files in os.walk(r"/home/agus/Documents/ITBA/POLIMI/Multidisciplinary Project/Pokemon-Strategy-using-Reinforcement-Learning/pokémonTeams/teams1"):
	for file in files:
		teamsPaths.append(os.path.join(root,file))

teams = []
for teamPath in teamsPaths:
    with open(teamPath, "r") as team:
        teams.append(team.read())


In [4]:
# print(teams)
custom_builder = RandomTeamFromPool(teams)

In [5]:
import subprocess
import json

def run_javascript(file_path, *args):
    result = subprocess.run(["node", file_path] + list(args), capture_output=True, text=True, check=True)
    return json.loads(result.stdout)


In [6]:
from poke_env.player.player import Player
from poke_env.environment.abstract_battle import AbstractBattle

# Define a custom player by subclassing Player
class CustomPlayer(Player):
    def choose_move(self, battle: AbstractBattle):
        if battle.available_moves:
            return self.create_order(max(battle.available_moves, key=lambda move: move.base_power*move.accuracy))
        else:
            return self.choose_random_move(battle)

# Define a optimized player by subclassing Player
class OptimizedPlayer(Player):                
    def choose_move(self, battle: AbstractBattle):
        # print(f"Own pokemon: {battle.active_pokemon}")
        # print(f"Own pokemon name: {battle.active_pokemon.species}")
        # print(f"Own pokemon level: {battle.active_pokemon.level}")
        # print(f"Own pokemon item: {battle.active_pokemon.item}")
        # print(f"Own pokemon status: {battle.active_pokemon.status}")
        # print(f"Own pokemon ability: {battle.active_pokemon.ability}")
        # print(f"Own pokemon boosts: {battle.active_pokemon.boosts}")
        
        # print(f"Own pokemon stats: {battle.active_pokemon.stats}")
        # print(f"Own pokemon base stats: {battle.active_pokemon.base_stats}")
        
        # print(f"Opponent pokemon: {battle.opponent_active_pokemon}")
        # print(f"Opponent pokemon name: {battle.opponent_active_pokemon.species}")
        
        # print()
        if battle.available_moves:
            attacker = battle.active_pokemon
            attacker_name = str(attacker.species)
            attacker_level = str(attacker.level)
            attacker_item = str(attacker.item)
            attacker_status = str(attacker.status)
            attacker_ability = str(attacker.ability)
            attacker_boosts = str(attacker.boosts)
            
            defender = battle.opponent_active_pokemon
            defender_name = str(defender.species)
            defender_level = str(defender.level)
            defender_item = str(defender.item)
            defender_status = str(defender.status)
            defender_ability = str(defender.ability)
            defender_boosts = str(defender.boosts)
            
            average_damage_moves = {}
            for current_move in battle.available_moves:
            
                calculate_damage = run_javascript("damage_calculation.js", 
                attacker_name, attacker_level, attacker_item, attacker_status, attacker_ability, attacker_boosts, 
                defender_name, defender_level, defender_item, defender_status, defender_ability, defender_boosts, 
                               str(current_move.id))
                
                # print(f"Current moves: {current_move}")
                # print(f"Current move damage: {calculate_damage}")
                
                if calculate_damage == 0:
                    average_damage_moves[current_move] = 0
                else:
                    average_damage_moves[current_move] = sum(calculate_damage)/len(calculate_damage)
                    
            # print(f"Current moves: {average_damage_moves}")
            # print(f"Picked move: {max(average_damage_moves, key=average_damage_moves.get)}")
            return self.create_order(max(average_damage_moves, key=average_damage_moves.get))
        else:
            return self.choose_random_move(battle)



In [7]:
from poke_env import RandomPlayer
from poke_env import LocalhostServerConfiguration

server_configuration = LocalhostServerConfiguration
generation = "gen2ou"

optimized_player = OptimizedPlayer(
    server_configuration=server_configuration,
    battle_format=generation,
    team=custom_builder)

custom_player = CustomPlayer(
    server_configuration=server_configuration,
    battle_format=generation,
    team=custom_builder)

random_player = RandomPlayer(
    server_configuration=server_configuration,
    battle_format=generation,
    team=custom_builder)

random_player2 = RandomPlayer(
    server_configuration=server_configuration,
    battle_format=generation,
    team=custom_builder)


In [8]:
import time
start_time = time.time()
await optimized_player.battle_against(custom_player, n_battles=1)
print("--- %s seconds ---" % (time.time() - start_time))

--- 12.967661380767822 seconds ---


In [10]:
print(f"Battles played: {optimized_player.n_finished_battles}")
print(f"Battles won: {optimized_player.n_won_battles}")
print(f"Win rate: {optimized_player.win_rate}")

Battles played: 1
Battles won: 0
Win rate: 0.0


In [11]:
import pandas as pd

# Define the columns for the DataFrame
# columns = ['battle_number', 'battle_turn', 'side_conditions', 'opponent_side_conditions', 'weather', 'fields', 'active_pokemon', 'opponent_active_pokemon', 'events']
columns = ['battle_number', 'battle_turn', 'active_pokemon', 'opponent_active_pokemon', 'events']


def parse_battle_observations():
    battle_results = []
    for battle_number, battle in enumerate(optimized_player.battles.values(), start=1):
        observations = battle.observations
        # print(observations)

        for turn, observation in observations.items():
            # side_conditions = observation.side_conditions
            # opponent_side_conditions = observation.opponent_side_conditions
            # weather = observation.weather
            # fields = observation.fields
            active_pokemon = observation.active_pokemon.species if observation.active_pokemon else None
            opponent_active_pokemon = observation.opponent_active_pokemon.species if observation.opponent_active_pokemon else None
            events = observation.events

            battle_results.append({
                'battle_number': battle_number,
                'battle_turn': turn,
                # 'side_conditions': side_conditions,
                # 'opponent_side_conditions': opponent_side_conditions,
                # 'weather': weather,
                # 'fields': fields,
                'active_pokemon': active_pokemon,
                'opponent_active_pokemon': opponent_active_pokemon,
                'events': events
            })
    return battle_results

# Parse the battle observations and create a DataFrame
battle_results = parse_battle_observations()
df = pd.DataFrame(battle_results, columns=columns)

# Export DataFrame to JSON
df.to_json('results/battle_results.json', orient='records')
print(df)



    battle_number  battle_turn active_pokemon opponent_active_pokemon  \
0               1            0           None                    None   
1               1            1         zapdos                smeargle   
2               1            2         zapdos                smeargle   
3               1            3         zapdos                smeargle   
4               1            4         zapdos                smeargle   
5               1            5         zapdos                smeargle   
6               1            6         zapdos                 machamp   
7               1            7         zapdos                 machamp   
8               1            8         zapdos                 snorlax   
9               1            9         zapdos                 snorlax   
10              1           10         gengar                 snorlax   
11              1           11      exeggutor                  scizor   
12              1           12        snorlax      

In [1]:
import asyncio

async def main():
    await optimized_player.battle_against(custom_player, n_battles=1)

if __name__ == "__main__":
    asyncio.get_event_loop().run_until_complete(main())

RuntimeError: This event loop is already running

In [68]:
import itertools

# Define natures and their stat changes
natures = {
    'Hardy': {'atk': 1.0, 'def': 1.0, 'spa': 1.0, 'spd': 1.0, 'spe': 1.0},
    'Lonely': {'atk': 1.1, 'def': 0.9, 'spa': 1.0, 'spd': 1.0, 'spe': 1.0},
    'Brave': {'atk': 1.1, 'def': 1.0, 'spa': 1.0, 'spd': 1.0, 'spe': 0.9},
    'Adamant': {'atk': 1.1, 'def': 1.0, 'spa': 0.9, 'spd': 1.0, 'spe': 1.0},
    'Naughty': {'atk': 1.1, 'def': 1.0, 'spa': 1.0, 'spd': 0.9, 'spe': 1.0},
    'Bold': {'atk': 0.9, 'def': 1.1, 'spa': 1.0, 'spd': 1.0, 'spe': 1.0},
    'Docile': {'atk': 1.0, 'def': 1.0, 'spa': 1.0, 'spd': 1.0, 'spe': 1.0},
    'Relaxed': {'atk': 1.0, 'def': 1.1, 'spa': 1.0, 'spd': 1.0, 'spe': 0.9},
    'Impish': {'atk': 1.0, 'def': 1.1, 'spa': 0.9, 'spd': 1.0, 'spe': 1.0},
    'Lax': {'atk': 1.0, 'def': 1.1, 'spa': 1.0, 'spd': 0.9, 'spe': 1.0},
    'Timid': {'atk': 0.9, 'def': 1.0, 'spa': 1.0, 'spd': 1.0, 'spe': 1.1},
    'Hasty': {'atk': 1.0, 'def': 0.9, 'spa': 1.0, 'spd': 1.0, 'spe': 1.1},
    'Serious': {'atk': 1.0, 'def': 1.0, 'spa': 1.0, 'spd': 1.0, 'spe': 1.0},
    'Jolly': {'atk': 1.0, 'def': 1.0, 'spa': 0.9, 'spd': 1.0, 'spe': 1.1},
    'Naive': {'atk': 1.0, 'def': 1.0, 'spa': 1.0, 'spd': 0.9, 'spe': 1.1},
    'Modest': {'atk': 0.9, 'def': 1.0, 'spa': 1.1, 'spd': 1.0, 'spe': 1.0},
    'Mild': {'atk': 1.0, 'def': 0.9, 'spa': 1.1, 'spd': 1.0, 'spe': 1.0},
    'Quiet': {'atk': 1.0, 'def': 1.0, 'spa': 1.1, 'spd': 1.0, 'spe': 0.9},
    'Bashful': {'atk': 1.0, 'def': 1.0, 'spa': 1.0, 'spd': 1.0, 'spe': 1.0},
    'Rash': {'atk': 1.0, 'def': 1.0, 'spa': 1.1, 'spd': 0.9, 'spe': 1.0},
    'Calm': {'atk': 0.9, 'def': 1.0, 'spa': 1.0, 'spd': 1.1, 'spe': 1.0},
    'Gentle': {'atk': 1.0, 'def': 0.9, 'spa': 1.0, 'spd': 1.1, 'spe': 1.0},
    'Sassy': {'atk': 1.0, 'def': 1.0, 'spa': 1.0, 'spd': 1.1, 'spe': 0.9},
    'Careful': {'atk': 1.0, 'def': 1.0, 'spa': 0.9, 'spd': 1.1, 'spe': 1.0},
    'Quirky': {'atk': 1.0, 'def': 1.0, 'spa': 1.0, 'spd': 1.0, 'spe': 1.0}
}

# Function to calculate a stat
def calculate_stat(base_stat, iv, ev, level, nature_modifier):
    return int((((2 * base_stat + iv + (ev // 4)) * level) // 100 + 5) * nature_modifier)

# Function to estimate IVs, EVs, and Nature for all stats
def estimate_all_stats(base_stats, observed_stats, level):
    possible_results = []

    # Iterate over all possible IVs (0-31) and EVs (0-252, step by 4)
    for ivs in itertools.product(range(32), repeat=5):
        for evs in itertools.product(range(0, 253, 4), repeat=5):
            for nature, modifiers in natures.items():
                matched = True
                for stat_key, base_stat in base_stats.items():
                    observed_stat = observed_stats[stat_key]
                    iv = ivs[list(base_stats.keys()).index(stat_key)]
                    ev = evs[list(base_stats.keys()).index(stat_key)]
                    nature_modifier = modifiers[stat_key]
                    calculated_stat = calculate_stat(base_stat, iv, ev, level, nature_modifier)

                    if calculated_stat != observed_stat:
                        matched = False
                        break

                if matched:
                    possible_results.append({
                        'ivs': ivs,
                        'evs': evs,
                        'nature': nature
                    })

    return possible_results

# Example Usage
# Assuming Haunter's base stats in Gen 8 are as follows:
base_stats = {
    'atk': 50,
}

# Observed stats at level 50
observed_stats = {
    'atk': 70,
}

level = 50

# Estimate possible matches
possible_matches = estimate_all_stats(base_stats, observed_stats, level)

# Print possible matches
for match in possible_matches:
    ivs = match['ivs']
    evs = match['evs']
    nature = match['nature']
    print(f"IVs: {ivs}, EVs: {evs}, Nature: {nature}")


KeyboardInterrupt: 

In [4]:
def run_javascript(file_path, *args):
    result = subprocess.run(["node ", file_path] + list(args), capture_output=True, text=True, check=True)
    return result.stdout

# run_javascript("test.js", )
damage_calculator_js = "damage_calculation.js"
attacker = "Gengar"
defender = "Chansey"
move = "Thunderbolt"

damage = run_javascript(damage_calculator_js, attacker, defender, move)

if damage:
    print("Calculated damage:", damage)
    average_damage = sum(damage)/len(damage)
    print(f"Average damage: {average_damage}")
else:
    print("Could not calculate damage.")


In [108]:
arg1 = 'Gengar'
arg2 = '100'
arg3 = 'leftovers'
arg4 = 'None'
arg5 = 'noability'
arg6 = "{'accuracy': 0, 'atk': 0, 'def': 0, 'evasion': 0, 'spa': 0, 'spd': 0, 'spe': 0}"

arg7 = 'Chansey'
arg8 = '100'
arg9 = 'leftovers'
arg10 = 'None'
arg11 = 'noability'
arg12 = "{'accuracy': 0, 'atk': 0, 'def': 0, 'evasion': 0, 'spa': 0, 'spd': 0, 'spe': 0}"

arg13 = 'Thunderbolt'

# damage_calc = run_javascript("damage_calculation.js", 
# attacker_name, attacker_level, attacker_item, attacker_status, attacker_ability, attacker_boosts, 
# defender_name, defender_level, defender_item, defender_status, defender_ability, defender_boosts, 
#                str(move))

damage_calc = run_javascript("damage_calculation.js", arg1, arg2, arg3, arg4, arg5, arg6, arg7, arg8, arg9, arg10, arg11, arg12, arg13)
print(damage_calc)

[79, 80, 80, 81, 81, 81, 82, 82, 82, 83, 83, 84, 84, 84, 85, 85, 85, 86, 86, 86, 87, 87, 88, 88, 88, 89, 89, 89, 90, 90, 91, 91, 91, 92, 92, 92, 93, 93, 94]
